# Установка библиотек

In [1]:
import pandas as pd #для работы с таблицами

Для автоматического применения статистического критерия (теста) используется модуль `stats` библиотеки `scipy`.

Этот модуль содержит большое количество вероятностных распределений, корреляционные функции и статистические тесты и многое другое.

Также активно используется библиотека `statsmodels`

Для установки библиотеки `scipy` и `statsmodels` можно воспользоваться менеджером пакетов `pip`.

In [2]:
%pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install statsmodels

Note: you may need to restart the kernel to use updated packages.



# Загрузка данных

Для скачивания данных используем команду `!wget`. 

Данные появятся в локальной директории.

Мы будем использовать информацию о диаметре пицц разных производителей.

In [4]:
!wget https://raw.githubusercontent.com/harika-bonthu/Hypothesis-test-examples/main/pizzas.csv

--2023-05-30 22:22:48--  https://raw.githubusercontent.com/harika-bonthu/Hypothesis-test-examples/main/pizzas.csv
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 539 [text/plain]
Сохранение в: «pizzas.csv.4»

pizzas.csv.4        100%[===================>]     539  --.-KB/s    за 0s      

2023-05-30 22:22:48 (17,7 MB/s) - «pizzas.csv.4» сохранён [539/539]



Загрузим данные в память ноутбука.

In [5]:
data = pd.read_csv('pizzas.csv')

In [6]:
data.rename(columns={'Making Unit 1': 'Пиццерия №1', 'Making Unit 2': 'Пиццерия №2'}, inplace=True)

In [7]:
data.head()

,Пиццерия №1,Пиццерия №2
0,6.8090,6.7703
1,6.4376,7.5093
2,6.9157,6.7300
3,7.3012,6.7878
4,7.4488,7.1522


# Проверка данных на нормальность


In [8]:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'

Установим уровень значимости

In [9]:
alpha = 0.05

## Тест Шапиро-Уилка

In [10]:
from scipy.stats import shapiro

In [11]:
_, p = shapiro(data)
print('p=%.3f' % p)

# Интерпретация 

if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.204
Данные распределены нормально


## Тест Д’Агостино

In [12]:
from scipy.stats import normaltest

In [13]:
_, p = normaltest(data)
#normaltest возвращает двустороннюю вероятность для проверки гипотезы

print('p=%.3f' % p[0])

# Интерпретация 

if p[0] > alpha/2:
	print(H0)
else:
	print(Ha)

p=0.251
Данные распределены нормально


# Независимый T-тест 

In [14]:
from scipy.stats import ttest_ind

In [15]:
H0 = 'Нет значимой разницы между диаметрами пиццы в разных пиццериях.'
Ha = 'Есть значимая разница между диаметрами пиццы в разных пиццериях.'

Зависимая переменная (диаметр пиццы) является количественной. Группы происходят из разных совокупностей. Следовательно, мы используем независимый T-тест.

In [16]:
def t_test(df):
    print('\n' + "*** Результаты независимого T-теста ***")
    test_results = ttest_ind(data['Пиццерия №1'], data['Пиццерия №2'], equal_var=True)

    p = round(test_results[1],2)

    if p>alpha:
        print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
    else:
        print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

t_test(data)


*** Результаты независимого T-теста ***
0.47 > 0.05. Мы не можем отвергнуть нулевую гипотезу. Нет значимой разницы между диаметрами пиццы в разных пиццериях.


-----

# Корреляция Спирмана 

In [17]:
from numpy.random import rand
from scipy.stats import spearmanr

data1 = rand(1000) * 20
data2 = data1 + (rand(1000) * 10)


corr, p = spearmanr(data1, data2)

In [18]:
print(corr,p)

0.9093775653775653 0.0


In [19]:
if p>alpha:
    print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу об отсутсвии зависимости между переменными.")
else:
    print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу об отсутсвии зависимости между переменными.")

0.0 <= 0.05. Мы отвергаем нулевую гипотезу об отсутсвии зависимости между переменными.


# ANOVA тест

In [20]:
from scipy.stats import f_oneway

Однофакторный дисперсионный анализ ANOVA проверяет нулевую гипотезу о том, что две или более групп имеют одинаковое среднее значение генеральной совокупности.

В качестве данных возьмем информацию по размерам раковины мидий, выращенных в разных местах.

In [21]:
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764,
           0.0689]
tvarminne = [0.0703, 0.1026, 0.0956, 0.0973, 0.1039, 0.1045]

In [22]:
_, p = f_oneway(petersburg, magadan, tvarminne)

In [23]:
H0 = 'Нет значимой разницы между средним размером раковины мидий в трех разных местах.'
Ha = 'Есть значимая разница между средним размером раковины мидий в трех разных местах.'

In [24]:
if p>alpha:
  print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
else:
  print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

0.007870900547143829 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница между средним размером раковины мидий в трех разных местах.


_____

# Домашнее задание

Используя информацию по мидиям Петербурга и Магадана, которые представлены в виде двух массивов petersburg и magadan в разделе ANOVA тест, проверьте данные на нормальность и на наличие корреляции. Для этого объедините 2 массива в DataFrame. Обоснуйте выбор теста на корреляцию. Сделайте вывод по гипотезе.
 
 
Также ответьте на вопрос:
Какова разница в среднем размере мидии в зависимости от города-производителя. Обоснуйте выбор теста. Сделайте вывод по гипотезе.


In [25]:
import statistics

In [26]:
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764,
           0.0689]

In [27]:
#    Поскольку разная длина массивов, дополним более короткий медианным значением

display(petersburg)
petersburg.append(statistics.median(petersburg))
display(petersburg)

[0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]

[0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105, 0.1016]

In [28]:
#    объединяем 2 массива в единый DataFrame

data_m = pd.DataFrame({'Петербург': petersburg,
                       'Магадан': magadan})
data_m

,Петербург,Магадан
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,0.1016,0.0689


In [29]:
#    проверим массив Петербург на нормальность распределения тестом Шапиро-Уилка

H0 = 'Данные "Петербург" распределены нормально'
Ha = 'Данные "Петербург" не распределены нормально (мы отвергаем H0)'

alpha = 0.05

_, p = shapiro(data_m['Петербург'])
print('p=%.3f' % p)

# Интерпретация 

if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.106
Данные "Петербург" распределены нормально


In [30]:
#    проверим массив Магадан на нормальность распределения тестом Шапиро-Уилка

H0 = 'Данные "Магадан" распределены нормально'
Ha = 'Данные "Магадан" не распределены нормально (мы отвергаем H0)'

alpha = 0.05

_, p = shapiro(data_m['Магадан'])
print('p=%.3f' % p)

# Интерпретация 

if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.036
Данные "Магадан" не распределены нормально (мы отвергаем H0)


In [43]:
# Поскольку не все сравниваемые ряды нормально распределены, и обе переменные количественные, 
# используем для оценки корреляции метод Спирмана

from scipy.stats import spearmanr

alpha = 0.05

corr, p = spearmanr(data_m)

print(corr, p)



0.15569141404872366 0.7127617079991629


In [44]:
if p>alpha:
    print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу об отсутсвии зависимости между переменными.")
else:
    print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу об отсутсвии зависимости между переменными.")

0.7127617079991629 > 0.05. Мы не можем отвергнуть нулевую гипотезу об отсутсвии зависимости между переменными.


Полученные результаты говорят о том, что данные о размере раковин мидий в Петербурге 
и Магадане не связаны между собой. Гипотеза о наличии значимой связи не подтвердилась.

Для выбора метода сравнения средних значений размера раковин мидий в Петербурге и Магадане
воспользуемся следующим алгоритмом:
- мы сравниваем количественные признаки
- имеем 2 группы значений
- группы независимы, т.к. в данном случае расположены в географически разных местах
- условие нормального распределения сравниваемых данных не соблюдается
Следовательно, целесообразно применить U-критерий Манна-Уитни.

In [50]:
import scipy.stats as stats

H0 = 'Средний размер мидий не зависит от региона'
Ha = 'Средний размер мидий зависит от региона'

alpha = 0.05

_,p = stats.mannwhitneyu(data_m['Магадан'], data_m['Петербург'])
print(p)

if p<=alpha:
    print('Вывод: ',Ha)
else:
    print('Вывод: ',H0)

0.007362062522639361
Вывод:  Средний размер мидий зависит от региона


# Бонус

Скачаем датасет, содержащий информацию по давлению 120 человек.



In [33]:
!wget https://raw.githubusercontent.com/yug95/MachineLearning/master/Hypothesis%20testing/blood_pressure.csv

--2023-05-30 22:22:53--  https://raw.githubusercontent.com/yug95/MachineLearning/master/Hypothesis%20testing/blood_pressure.csv
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 2851 (2,8K) [text/plain]
Сохранение в: «blood_pressure.csv.3»

blood_pressure.csv. 100%[===================>]   2,78K  --.-KB/s    за 0s      

2023-05-30 22:22:54 (18,6 MB/s) - «blood_pressure.csv.3» сохранён [2851/2851]



In [34]:
data = pd.read_csv('blood_pressure.csv')

In [35]:
print('Размер выборки: ', data.shape)

Размер выборки:  (120, 5)


In [36]:
data.head()

,patient,sex,agegrp,bp_before,bp_after
0,1,Male,30-45,143,153
1,2,Male,30-45,163,170
2,3,Male,30-45,153,168
3,4,Male,30-45,153,142
4,5,Male,30-45,146,141


## Z-тест

Проведем Z-тест для проверки двух независимых групп данных на равенство средних значений выборок.

In [37]:
from statsmodels.stats import weightstats

In [38]:
_ ,p = weightstats.ztest(data['bp_before'], x2=data['bp_after'], value=0,alternative='two-sided')
print(float(p))


if p < 0.05:
    print("Мы отвергаем нулевую гипотезу об отсутсвии различий между данными.")
else:
    print("Мы не можем отвергнуть нулевую гипотезу об отсутсвии различий между данными.")

0.002162306611369422
Мы отвергаем нулевую гипотезу об отсутсвии различий между данными.


## Z-тест на пропорцию

Z-критерий одной пропорции используется для сравнения наблюдаемой пропорции с теоретической.

В этом тесте используются следующие нулевые гипотезы:

$H_0: p = p_0$ (доля мужчин, предоставивших данные о своем давлении, равна гипотетической пропорции $p_0$)

Альтернативная гипотеза может быть двусторонней, левосторонней или правосторонней:

$H_1 (двусторонний): p ≠ p_0$ (доля мужчин не равна некоторому гипотетическому значению $p_0$)
$H_1 (левосторонний): p < p_0$ (доля мужчин меньше некоторого гипотетического значения $p_0$)
$H_1 (правосторонний): p > p_0$ (доля мужчин больше некоторого гипотетического значения $p_0$)

Предположим, что доля мужчин в нашем датачете равна 40%.

$p_0$: гипотетическая доля мужчин = 0.40

$x$: число мужчин в выборке мужчин: `len(data[data.sex == 'Male'])`

$n$: размер выборки = `len(data)`

Покажем, как использовать функцию `proportions_ztest` для выполнения  **z-теста**:

In [39]:
p_0 = 0.4
n = len(data)
x = len(data[data.sex == 'Male'])

print(n, x)

120 60


In [40]:
from statsmodels.stats.proportion import proportions_ztest

#perform one proportion z-test
_, p = proportions_ztest(count=x, nobs=n, value=p_0)

In [41]:
if p < 0.05:
    print("Мы отвергаем нулевую гипотезу о том, что доля мужчин, предоставивших данные о своем давлении, равна 40%.")
else:
    print("Мы не можем отвергнуть нулевую гипотезу.")

Мы отвергаем нулевую гипотезу о том, что доля мужчин, предоставивших данные о своем давлении, равна 40%.
